In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import glob, os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [133]:
mask_dir = f'/global/cfs/cdirs/m1867/gsharing/MCSMIP/DYAMOND/mcs_mask/'
mask_file = f'{mask_dir}mcs_mask_IFS.nc'

orig_statsfile = '/pscratch/sd/f/feng045/DYAMOND/Summer/IFS/stats/mcs_tracks_final_20160801.0000_20160910.0000.nc'
print(mask_file)

/global/cfs/cdirs/m1867/gsharing/MCSMIP/DYAMOND/mcs_mask/mcs_mask_IFS.nc


In [134]:
dsstats = xr.open_dataset(orig_statsfile)
base_time_orig = dsstats.base_time

In [3]:
ds = xr.open_dataset(mask_file)
ds

<xarray.Dataset>
Dimensions:     (time: 959, lat: 1200, lon: 3600)
Coordinates:
  * time        (time) datetime64[ns] 2016-08-01T01:00:00 ... 2016-09-09T23:0...
  * lat         (lat) float32 -59.95 -59.85 -59.75 -59.65 ... 59.75 59.85 59.95
  * lon         (lon) float32 -179.9 -179.9 -179.8 -179.6 ... 179.8 179.9 179.9
Data variables:
    cloud_mask  (time, lat, lon) int64 ...
    mcs_mask    (time, lat, lon) int64 ...
Attributes:
    Title:       Example MCS mask file
    Contact:     Zhe Feng, zhe.feng@pnnl.gov
    Created_on:  Fri Aug  4 10:07:59 2023

In [4]:
mcs_mask = ds.mcs_mask.load()

In [107]:
ntimes = ds.sizes['time']

# Get max track number, this is the total number of tracks
ntracks = mcs_mask.max().item()
print(f'Total number of times: {ntimes}')
print(f'Total number of tracks: {ntracks}')

Total number of times: 959
Total number of tracks: 4169


In [108]:
np.unique(mcs_mask.isel(time=0))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38])

In [102]:
# Find all unique track numbers at each time and save to a list
# This should take ~1 min to run for 960 times
list_mcs = []
for itime in range(0, ntimes):
    # print(itime)
    list_mcs.append(np.unique(mcs_mask.isel(time=itime)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [109]:
len(list_mcs)

959

In [110]:
list_mcs[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38])

In [111]:
def find_track_time_indices(list_tracks, track_number):

    # Initialize a list to store the indices of lists containing the track number
    indices_with_track = []

    # Iterate through the lists and find the indices where the track number is present
    for i, lst in enumerate(list_tracks):
        if track_number in lst:
            indices_with_track.append(i)

    return indices_with_track

In [129]:
# Create an empty datetime64[ns] array filled with NaT (Not a Time)
dims = (ntracks, 500)
base_times = np.full(dims, np.datetime64('NaT'), dtype='datetime64[ns]')

for tracknum in range(1, ntracks+1):
    ii = tracknum - 1  # track index
    # Find time indices for the track
    tidx_tracknum = find_track_time_indices(list_mcs, tracknum)
    if tidx_tracknum:
        base_times[ii, :len(tidx_tracknum)] = ds.time.isel(time=tidx_tracknum).data

In [130]:
base_times[0,0:100]

array(['2016-08-01T01:00:00.000000000', '2016-08-01T02:00:00.000000000',
       '2016-08-01T03:00:00.000000000', '2016-08-01T04:00:00.000000000',
       '2016-08-01T05:00:00.000000000', '2016-08-01T06:00:00.000000000',
       '2016-08-01T07:00:00.000000000', '2016-08-01T08:00:00.000000000',
       '2016-08-01T09:00:00.000000000', '2016-08-01T10:00:00.000000000',
       '2016-08-01T11:00:00.000000000', '2016-08-01T12:00:00.000000000',
       '2016-08-01T13:00:00.000000000', '2016-08-01T14:00:00.000000000',
       '2016-08-01T15:00:00.000000000', '2016-08-01T16:00:00.000000000',
       '2016-08-01T17:00:00.000000000', '2016-08-01T18:00:00.000000000',
       '2016-08-01T19:00:00.000000000', '2016-08-01T20:00:00.000000000',
       '2016-08-01T21:00:00.000000000', '2016-08-01T22:00:00.000000000',
       '2016-08-01T23:00:00.000000000', '2016-08-02T00:00:00.000000000',
       '2016-08-02T01:00:00.000000000', '2016-08-02T02:00:00.000000000',
       '2016-08-02T03:00:00.000000000', '2016-08-02

In [132]:
base_times[-1,0:10]

array(['2016-09-09T19:00:00.000000000', '2016-09-09T20:00:00.000000000',
       '2016-09-09T21:00:00.000000000', '2016-09-09T22:00:00.000000000',
       '2016-09-09T23:00:00.000000000',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT'],
      dtype='datetime64[ns]')

In [135]:
base_time_orig.shape, base_times.shape

((4169, 500), (4169, 500))

In [141]:
# Compare the base_time from the stats file with the reconstructed base_time from MCS mask files
# They should be identical
bt_diff = base_time_orig.data - base_times
np.nanmax(bt_diff)

numpy.timedelta64(0,'ns')

In [142]:
bt_diff

array([[    0,     0,     0, ..., 'NaT', 'NaT', 'NaT'],
       [    0,     0,     0, ..., 'NaT', 'NaT', 'NaT'],
       [    0,     0,     0, ..., 'NaT', 'NaT', 'NaT'],
       ...,
       [    0,     0,     0, ..., 'NaT', 'NaT', 'NaT'],
       [    0,     0,     0, ..., 'NaT', 'NaT', 'NaT'],
       [    0,     0,     0, ..., 'NaT', 'NaT', 'NaT']],
      dtype='timedelta64[ns]')

In [143]:
# plt.hist(bt_diff.data)

In [94]:
itrack = 100
tidx_itrack = find_track_time_indices(list_mcs, itrack)
print(tidx_itrack)

[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [92]:
if tidx_itrack = find_track_time_indices(list_mcs, itrack):
    # Create an empty datetime64[ns] array filled with NaT (Not a Time)
    itrack_bt = np.full(50, np.datetime64('NaT'), dtype='datetime64[ns]')
    # Assign valid times of the track to the array
    itrack_bt[:len(tidx_itrack)] = ds.time.isel(time=tidx_itrack).data
itrack_bt

array(['2016-08-01T10:00:00.000000000', '2016-08-01T11:00:00.000000000',
       '2016-08-01T12:00:00.000000000', '2016-08-01T13:00:00.000000000',
       '2016-08-01T14:00:00.000000000', '2016-08-01T15:00:00.000000000',
       '2016-08-01T16:00:00.000000000', '2016-08-01T17:00:00.000000000',
       '2016-08-01T18:00:00.000000000', '2016-08-01T19:00:00.000000000',
       '2016-08-01T20:00:00.000000000', '2016-08-01T21:00:00.000000000',
       '2016-08-01T22:00:00.000000000', '2016-08-01T23:00:00.000000000',
       '2016-08-02T00:00:00.000000000', '2016-08-02T01:00:00.000000000',
       '2016-08-02T02:00:00.000000000', '2016-08-02T03:00:00.000000000',
       '2016-08-02T04:00:00.000000000', '2016-08-02T05:00:00.000000000',
       '2016-08-02T06:00:00.000000000', '2016-08-02T07:00:00.000000000',
       '2016-08-02T08:00:00.000000000', '2016-08-02T09:00:00.000000000',
       '2016-08-02T10:00:00.000000000', '2016-08-02T11:00:00.000000000',
       '2016-08-02T12:00:00.000000000', '2016-08-02

In [93]:
# # Create an empty datetime64[ns] array filled with NaT (Not a Time)
# itrack_bt = np.full(50, np.datetime64('NaT'), dtype='datetime64[ns]')
# # Assign valid times of the track to the array
# itrack_bt[:len(tidx_itrack)] = ds.time.isel(time=tidx_itrack).data
# itrack_bt